In [ ]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_terminal = False
        self.path = None
        self.max_distance =  float('inf')   # Initial Levenshtein threshold

def levenshtein_distance(word1, word2):
    len1 = len(word1) + 1
    len2 = len(word2) + 1

    # Create the distance matrix
    dp = [[0 for _ in range(len2)] for _ in range(len1)]

    # Initialize base cases
    for i in range(len1):
        dp[i][0] = i
    for j in range(len2):
        dp[0][j] = j

    # Calculate minimum edit operations
    for i in range(1, len1):
        for j in range(1, len2):
            if word1[i - 1] == word2[j - 1]:
                cost = 0  # Substitution cost if characters match
            else:
                cost = 1  # Substitution cost

            dp[i][j] = min(
                dp[i - 1][j] + 1,     # Deletion 
                dp[i][j - 1] + 1,     # Insertion
                dp[i - 1][j - 1] + cost  # Substitution
            )

    return dp[len1 - 1][len2 - 1]  # Result at the bottom right of the matrix

class AppLayoutTrie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, path):
        current_node = self.root
        for word in path.split(" > "):
            current_node = current_node.children.setdefault(word, TrieNode())
        current_node.is_terminal = True
        current_node.path = path
 
    def search(self, query, threshold=10):  # Default tolerance
        results = []

        def _dfs(node):
            
            if node.is_terminal:
                return

            for child_word, child_node in node.children.items():
                distance = levenshtein_distance(query, child_word)
                if distance <= threshold:
                    results.append(child_node.path)
                else:
                    _dfs(child_node)

                    
        _dfs(self.root)
        return results

# Construct the trie for testing
trie = AppLayoutTrie()
# trie.insert("select_post")
trie.insert("select_post > new_comment")
trie.insert("select_post")

print(trie.root.children['select_post'].children['new_comment'].path)
print(trie.root.children['select_post'].path)

# no problem in insert

In [45]:
def levenshtein_distance(word1, word2):
    len1 = len(word1) + 1
    len2 = len(word2) + 1

    # Create the distance matrix
    dp = [[0 for _ in range(len2)] for _ in range(len1)]

    # Initialize base cases
    for i in range(len1):
        dp[i][0] = i
    for j in range(len2):
        dp[0][j] = j

    # Calculate minimum edit operations
    for i in range(1, len1):
        for j in range(1, len2):
            if word1[i - 1] == word2[j - 1]:
                cost = 0  # Substitution cost if characters match
            else:
                cost = 1  # Substitution cost

            dp[i][j] = min(
                dp[i - 1][j] + 1,  # Deletion
                dp[i][j - 1] + 1,  # Insertion
                dp[i - 1][j - 1] + cost,  # Substitution
            )

    return dp[len1 - 1][len2 - 1]  # Result at the bottom right of the matrix


class TrieNode:
    def __init__(self):
        self.children = {}
        self.path = None


class AppTrie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, path):
        current_node = self.root
        if current_node.path is None:
            current_node.path = path.split(" > ")[0]

        increasing_path = ""
        for word in path.split(" > "):
            increasing_path = increasing_path + " > " + word if increasing_path else word
            if word not in current_node.children:
                current_node.children[word] = TrieNode()
            current_node.children[word].path = increasing_path
            current_node = current_node.children[word]

    def search(self, query, threshold=10):
        results = []

        def _dfs_query(node: TrieNode, query: str):
            if node.children == {}:
                return
            
            print("Searching", node.children)

            for child_word, child_node in node.children.items():
                distance = levenshtein_distance(query, child_word)
                print("Distance", distance, child_word, query)
                if distance <= threshold:
                    results.append(child_node.path)
                else :
                    _dfs_query(child_node, query)
        
        _dfs_query(self.root, query)
        return results
    
# Construct the trie for testing
trie = AppTrie()
trie.insert("select_post > new_comment > add_image")
trie.insert("settings > battery > toggle_battery_saver")
trie.insert("select_post > new_comment")
trie.insert("select_post")

print("output:", trie.search("comment_on_image", 10))

Searching {'select_post': <__main__.TrieNode object at 0x7f919f2277f0>, 'settings': <__main__.TrieNode object at 0x7f919f227f70>}
Distance 13 select_post comment_on_image
Searching {'new_comment': <__main__.TrieNode object at 0x7f919f225a80>}
Distance 13 new_comment comment_on_image
Searching {'add_image': <__main__.TrieNode object at 0x7f919f225930>}
Distance 10 add_image comment_on_image
Distance 12 settings comment_on_image
Searching {'battery': <__main__.TrieNode object at 0x7f919f224d60>}
Distance 15 battery comment_on_image
Searching {'toggle_battery_saver': <__main__.TrieNode object at 0x7f919f224df0>}
Distance 15 toggle_battery_saver comment_on_image
output: ['select_post > new_comment > add_image']


In [15]:
print(trie.root)

In [ ]:
# Test Cases from Your Description
test_cases = [
    {"input":"toggle_battery_saver","output" :"settings > battery > toggle_battery_saver"},
    {"input":"turn_on_battery_saver","output":"settings > battery > toggle_battery_saver"},
    {"input":"turn_of_battery_saver","output":"settings > battery > toggle_battery_saver"},
    {"input":"comment_on_image","output": "select_post > new_comment > add_image"},
    {"input":"comment_on_post","output": "select_post > new_comment"},
    {"input":"see_post","output": "select_post"}
]

# Run tests
for case in test_cases:
    result = trie.search(case["input"])
    assert result == [case["output"]], f"Failed for input: {case['input']}"